# 13.14 Tweet Sentiment Analysis 
* Political researchers might use during elections to understand how people feel about specific politicians and issues, and **how they're likely to vote**
* Companies might use to see what people are saying about their products and competitors’ products
* Script `sentimentlistener.py` checks sentiment on a specified topic for a specified number of tweets

In [ ]:
run sentimentlistener.py football 10

<hr style="height:2px; border:none; color:#AAA; background-color:#AAA;">

### Class `SentimentListener`: Imports 
* Import the keys.py file and the libraries used throughout the script

```python
# sentimentlisener.py
"""Script that searches for tweets that match a search string
and tallies the number of positive, neutral and negative tweets."""
import keys
import preprocessor as p 
import sys
from textblob import TextBlob
import tweepy
```

### Class `SentimentListener`: `__init__` Method
* Receives:
    * `bearer_token` for authentication
    * `sentiment_dict` dictionary in which we’ll keep track of the tweet sentiments
    * `topic` we’re searching for so we can ensure that it appears in the tweet text  
    * `limit` of tweets to process (not including the ones we eliminate)
* Each of these is stored in the current `SentimentListener` object (`self`)

```python
class SentimentListener(tweepy.StreamingClient):
    """Handles incoming Tweet stream."""

    def __init__(self, bearer_token, sentiment_dict, topic, limit=10):
        """Configure the SentimentListener."""
        self.sentiment_dict = sentiment_dict
        self.tweet_count = 0
        self.topic = topic
        self.TWEET_LIMIT = limit

        # set tweet-preprocessor to remove URLs/reserved words
        p.set_options(p.OPT.URL, p.OPT.RESERVED) 
        super().__init__(bearer_token, wait_on_rate_limit=True)
```

### Method `on_response `
* If the tweet is not a retweet (line 28):
    * Line 29 gets and cleans the tweet’s text 
    * Lines 32–33 skip the tweet if it does not contain `topic` in the tweet text
    * Lines 36–45 use a `TextBlob` to check the tweet’s sentiment and update the `sentiment_dict` accordingly
    * Line 48 gets the sender’s `username` from `response.includes['users']` — we’ll use an expansion to include this user object 
    * Line 49 prints the tweet text preceded by `+` for positive sentiment, a space for neutral sentiment or `-` for negative sentiment
    * Line 51 increments the `tweet_count`, and lines 54–55 check whether the app should disconnect from the tweet stream

```python
def on_response(self, response):
        """Called when Twitter pushes a new tweet to you."""

        # if the tweet is not a retweet
        if not response.data.text.startswith('RT'):
            text = p.clean(response.data.text) # clean the tweet

            # ignore tweet if the topic is not in the tweet text
            if self.topic.lower() not in text.lower():
                return

            # update self.sentiment_dict with the polarity
            blob = TextBlob(text)
            if blob.sentiment.polarity > 0:
                sentiment = '+'
                self.sentiment_dict['positive'] += 1 
            elif blob.sentiment.polarity == 0:
                sentiment = ' '
                self.sentiment_dict['neutral'] += 1 
            else:
                sentiment = '-'
                self.sentiment_dict['negative'] += 1 

            # display the tweet
            username = response.includes['users'][0].username
            print(f'{sentiment} {username}: {text}\n')

            self.tweet_count += 1 # track number of tweets processed

            # if TWEET_LIMIT is reached, terminate streaming
            if self.tweet_count == self.TWEET_LIMIT:
                self.disconnect()
```

### Main Application
* The main application is defined in the function `main` (lines 57–87; discussed after the code), which is called by lines 90–91 when you execute the file as a script
* `sentimentlistener.py` also can be imported into IPython or other modules to use class `SentimentListener` as we did with `TweetListener`

```python
def main():
    # get search term and number of tweets
    search_key = sys.argv[1]
    limit = int(sys.argv[2]) # number of tweets to tally

    # set up the sentiment dictionary
    sentiment_dict = {'positive': 0, 'neutral': 0, 'negative': 0}

    # create the StreamingClient subclass object
    sentiment_listener = SentimentListener(keys.bearer_token, 
        sentiment_dict, search_key, limit)

    # redirect sys.stderr to sys.stdout
    sys.stderr = sys.stdout

    # delete existing stream rules
    rules = sentiment_listener.get_rules().data
    rule_ids = [rule.id for rule in rules]
    sentiment_listener.delete_rules(rule_ids)    

    # create stream rule
    sentiment_listener.add_rules(
        tweepy.StreamRule(f'{search_key} lang:en'))

    # start filtering English tweets containing search_key
    sentiment_listener.filter(expansions=['author_id'])

    print(f'Tweet sentiment for "{search_key}"')
    print('Positive:', sentiment_dict['positive'])
    print(' Neutral:', sentiment_dict['neutral'])
    print('Negative:', sentiment_dict['negative'])

# call main if this file is executed as a script
if __name__ == '__main__':
    main()

* In `main`:
    * Lines 59–60 get the command-line arguments
    * Line 63 creates the `sentiment_dict` dictionary that keeps track of the tweet sentiments
    * Lines 66–67 create the `SentimentListener` 
    * Line 70 redirects the standard error stream to the standard output stream
    * Lines 73–75 delete any existing `StreamRule`s
    * Lines 78–79 create a new `StreamRule` that searches for English (`lang:en`) tweets that match the `search_key`
    * Line 82 starts the stream — `expansions` indicates that we’d like Twitter to include the tweet sender’s user object in the response
    * Lines 84–87 display the sentiment report

------
&copy;1992&ndash;2020 by Pearson Education, Inc. All Rights Reserved. This content is based on Chapter 5 of the book [**Intro to Python for Computer Science and Data Science: Learning to Program with AI, Big Data and the Cloud**](https://amzn.to/2VvdnxE).

DISCLAIMER: The authors and publisher of this book have used their 
best efforts in preparing the book. These efforts include the 
development, research, and testing of the theories and programs 
to determine their effectiveness. The authors and publisher make 
no warranty of any kind, expressed or implied, with regard to these 
programs or to the documentation contained in these books. The authors 
and publisher shall not be liable in any event for incidental or 
consequential damages in connection with, or arising out of, the 
furnishing, performance, or use of these programs.                  